In [1]:
import sys
sys.path.append("../")

In [2]:
import torch
import numpy as np
from samplers.HamiltonianMonteCarlo import HamiltonianMonteCarlo
from utils import get_ground_truth_bnn, evaluate_test_nll, get_param_dim
from tqdm import tqdm

device = torch.device('cpu')
torch.set_default_tensor_type('torch.FloatTensor')

/Users/wenlinchen/miniconda3/envs/rdgs/lib/python3.9/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [3]:
d_x=20
d_h=25
s_n=0.1
dim = get_param_dim(d_x, d_h)
x_train, x_test, y_train, y_test, param_true, energy = get_ground_truth_bnn(d_x, d_h, s_n)

print("true_nll:", evaluate_test_nll(param_true, x_test, y_test, d_x, d_h, s_n))

true_nll: -0.8763405084609985


In [4]:
# smapler seed
seed = 1

np.random.seed(seed)
torch.manual_seed(seed)


In [5]:
num_samples = 150
num_chains = 2    # to save time, we run 2 independent HMC samplers with the same initialization in parallel, each generating 75 samples
step_size = 5e-4
num_hmc_steps_per_sample = 5
num_leapfrog_steps_per_hmc_step = 1000  # 1000x5=5000 leapfrog steps per sample

param = torch.randn([num_chains, dim]).to(device)

hmc = HamiltonianMonteCarlo(param, energy, step_size, num_leapfrog_steps_per_hmc_step, device=device)

samples = []
with tqdm(range(num_samples//num_chains)) as t_bar:
    for _ in t_bar:
        for _ in range(num_hmc_steps_per_sample):
            param, acc_rate = hmc.sample()
        t_bar.set_postfix({"acc_rate": acc_rate})
        samples.append(param.detach())
        nll = evaluate_test_nll(torch.cat(samples, dim=0), x_test, y_test, d_x, d_h, s_n)
        t_bar.set_postfix({"nll": nll})

samples = torch.cat(samples, dim=0)
nll = evaluate_test_nll(samples, x_test, y_test, d_x, d_h, s_n)
    

100%|██████████| 75/75 [01:42<00:00,  1.37s/it, nll=0.807]     


In [6]:
print("NLL:", nll)

NLL: 0.8065794706344604
